In [14]:
import numpy as np
import random
import optuna
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score
import json
import os
from optuna.visualization import plot_optimization_history

In [15]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"

In [16]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

In [17]:
def get_trend_changes_report_dict(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4,
        output_dict=True,
        zero_division=0
    )

def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        digits=4
    )

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    Returns only the F1-score for trend change.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [18]:
# Cargar datos
train = pd.read_csv("../../../data/post_cleaning/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/post_cleaning/validation_set.csv", parse_dates=["date"])
test_set = pd.read_csv("../../../data/post_cleaning/test_set.csv", parse_dates=['date'])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values
X_test = test_set.drop(columns=["date", "target_trend"]).values
y_test = test_set["target_trend"].values

In [19]:
def objective(trial):
    set_seeds(SEED)
    n_estimators = trial.suggest_int("n_estimators", 50, 400)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    subsample = trial.suggest_float("subsample", 0.6, 1.0)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=SEED
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = trend_changes_true(y_val, y_pred)
    return score

In [20]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=50)

[I 2025-08-24 19:30:06,946] A new study created in memory with name: no-name-91c8aa25-3150-446d-bdb3-bf4e57a6d637


[I 2025-08-24 19:30:13,775] Trial 0 finished with value: 0.3291139240506329 and parameters: {'n_estimators': 117, 'learning_rate': 0.06447363728960653, 'max_depth': 5, 'subsample': 0.9141434334855076, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.3291139240506329.
[I 2025-08-24 19:30:37,781] Trial 1 finished with value: 0.34375 and parameters: {'n_estimators': 147, 'learning_rate': 0.11047391773518671, 'max_depth': 10, 'subsample': 0.9503730538968379, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.34375.
[I 2025-08-24 19:30:58,721] Trial 2 finished with value: 0.24444444444444444 and parameters: {'n_estimators': 289, 'learning_rate': 0.08457591696604633, 'max_depth': 5, 'subsample': 0.82447847442625, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.34375.
[I 2025-08-24 19:31:19,975] Trial 3 finished with value: 0.3157894736842105 and parameters: {'n_estimators': 321, 'learning_rate': 0.140715735

In [21]:
# Visualizar el historial de optimización
plot_optimization_history(study)

In [22]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de {SCORE}: {study.best_value:.4f}")
best_params = study.best_params

Mejores hiperparámetros encontrados:
{'n_estimators': 343, 'learning_rate': 0.06620879032719892, 'max_depth': 7, 'subsample': 0.9326966142193129, 'min_samples_split': 2, 'min_samples_leaf': 9}
Mejor score de f1-score: 0.4058


In [23]:
# Guardar los mejores hiperparámetros y su valor
history = []
if os.path.exists("best_hyperparams.json"):
    try:
        with open("best_hyperparams.json", "r") as f:
            history = json.load(f)
    except (json.JSONDecodeError, ValueError):
        history = []

# Guardar ambos en un solo diccionario
history.append({
    "params": study.best_params,
    "value": study.best_value
})
# Guardar el historial de hiperparámetros
with open("best_hyperparams.json", "w") as f:
    json.dump(history, f, indent=2)

In [16]:
# # Cargar historial de hiperparámetros y valores
# import json

# with open("best_hyperparams.json", "r") as f:
#     history = json.load(f)

# # Escoger el último (más reciente)
# best_params = history[-1]["params"]
# best_value = history[-1]["value"]

# # Si quieres ver todos:
# for i, entry in enumerate(history):
#     print(f"Hiperparámetros #{i+1}: {entry['params']}, Valor: {entry['value']}")

# # Si quieres escoger uno específico (por índice):
# # best_params = history[indice_que_quieras]["params"]
# # best_value = history[indice_que_quieras]["value"]

Hiperparámetros #1: {'n_estimators': 324, 'learning_rate': 0.029233637937789516, 'max_depth': 4, 'subsample': 0.6813368996054391, 'min_samples_split': 10, 'min_samples_leaf': 9}, Valor: 0.32558139534883723


In [28]:
# Entrenar modelo final con los mejores hiperparámetros
set_seeds(SEED)
final_model = GradientBoostingClassifier(
    n_estimators=best_params["n_estimators"],
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    subsample=best_params["subsample"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    random_state=SEED
)
final_model.fit(X_train, y_train)
# Obtener predicciones
y_val_pred = final_model.predict(X_val)
print("Gradient Boosting Classifier Trend Changes Score:\n", trend_changes_score(y_val, y_val_pred))

Gradient Boosting Classifier Trend Changes Score:
               precision    recall  f1-score   support

       False     0.9225    0.9189    0.9207       259
        True     0.4000    0.4118    0.4058        34

    accuracy                         0.8601       293
   macro avg     0.6612    0.6653    0.6632       293
weighted avg     0.8619    0.8601    0.8609       293



In [29]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val, y_val_pred, digits=4)
print("Gradient Boosting Classifier Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val, y_val_pred))

Gradient Boosting Classifier Report:
               precision    recall  f1-score   support

          -1     0.8288    0.9787    0.8976        94
           0     0.7619    0.3902    0.5161        41
           1     0.9383    0.9560    0.9470       159

    accuracy                         0.8844       294
   macro avg     0.8430    0.7750    0.7869       294
weighted avg     0.8787    0.8844    0.8711       294

Balanced accuracy: 0.774980716487213


In [30]:
y_pred_test = final_model.predict(X_test)
print("\nGradientBoosting trend_changes_score:\n", trend_changes_score(y_test, y_pred_test))


GradientBoosting trend_changes_score:
               precision    recall  f1-score   support

       False     0.9141    0.8897    0.9017       263
        True     0.2368    0.2903    0.2609        31

    accuracy                         0.8265       294
   macro avg     0.5755    0.5900    0.5813       294
weighted avg     0.8427    0.8265    0.8342       294



In [31]:
# --- Exportar y Comparar Métricas de Modelos (Validación) ---

# 1. Definir el nombre del modelo actual y el archivo de salida
model_name = 'GradientBoosting'
output_file = '../../../score_models/model_comparison_metrics.csv'

# 2. Calcular el reporte de clasificación estándar
# Usamos y_val_m y y_val_pred que están en la misma escala (0,1,2)
report_dict = classification_report(y_val, y_val_pred, output_dict=True, zero_division=0)
precision = report_dict['macro avg']['precision']
recall = report_dict['macro avg']['recall']
f1_score = report_dict['macro avg']['f1-score']


# 3. Calcular el reporte de cambio de tendencia
report = get_trend_changes_report_dict(y_val, y_val_pred)
trend_change_precision = report['True']['precision']
trend_change_recall = report['True']['recall']
trend_change_f1_score = report['True']['f1-score']

# 4. Organizar las nuevas métricas
new_metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'trend_change_precision': trend_change_precision,
    'trend_change_recall': trend_change_recall,
    'trend_change_f1_score': trend_change_f1_score
}

# 5. Cargar, actualizar y guardar el DataFrame de comparación
try:
    # Intentar cargar el archivo existente
    comparison_df = pd.read_csv(output_file, index_col='model')
    # Si existe, actualizar o añadir la fila para el modelo actual
    comparison_df.loc[model_name] = new_metrics
except FileNotFoundError:
    # Si no existe, crear un DataFrame nuevo directamente con los datos actuales
    comparison_df = pd.DataFrame([new_metrics], index=[model_name])

# Guardar el DataFrame actualizado en el CSV
comparison_df.to_csv(output_file, index_label='model')